In [3]:
from urllib.request import urlopen
from urllib.parse import quote_plus as qp #문자 아스키코드로 변환
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
import time
import os

In [2]:
!pip install selenium

In [4]:
#인터넷 창 띄우기(chrome driver에 각자 개별 로그인)
driver = webdriver.Chrome()
url = 'https://www.instagram.com/'
driver.get(url)

In [8]:
#검색어 태그로 인터넷 창 이동
search = '오스테리아57' #검색어
scroll_num = 500 #스크롤 횟수
scroll_pause_time=1.5 #스크롤 내릴때 이미지 로딩되는 시간 기다림


baseurl = 'https://www.instagram.com/explore/tags/'
url = baseurl + qp(search)
driver.get(url)
WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.CLASS_NAME,'v1Nh3.kIKUG._bz0w')))

<selenium.webdriver.remote.webelement.WebElement (session="a0a19056fd37c024edef57cd3c6d0eca", element="b491372e-eb86-49c2-8d45-41a5e0a32a6b")>

In [9]:
#스크롤 하기
#스크롤은 기다리기 귀찮으면 직접 한 후에 아래 코드 실행해도 됨
#인터넷 환경에 따라 이미지 로딩이 느릴때, 스크롤이 적게 실행될 수 있음
for _ in range(scroll_num):
    time.sleep(scroll_pause_time)
    last_height = driver.execute_script("return document.body.scrollHeight")
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(scroll_pause_time)
    new_height = driver.execute_script("return document.body.scrollHeight")
    if new_height == last_height:
        driver.execute_script(
            "window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(scroll_pause_time)
        new_height = driver.execute_script(
            "return document.body.scrollHeight")
        if new_height == last_height:
            break
        else:
            last_height = new_height
            continue

In [42]:
for _ in range(1000):
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(1)

    driver.execute_script("window.scrollTo(0, 700)")
    time.sleep(1)

KeyboardInterrupt: 

In [43]:
#원하는 만큼 스크롤이 된 채로 실행
#이미지 저장 폴더 생성
if search not in os.listdir(): os.mkdir(search) 
            
#파싱 후 이미지 저장
html = driver.page_source
soup = BeautifulSoup(html)
insta = soup.select('.v1Nh3.kIKUG._bz0w')
for i, pic in enumerate(insta):
    imgurl = pic.select_one('.KL4Bh').img['src'] #이미지 주소 가져오기
    with urlopen(imgurl) as f: #날짜 시간 포함하여 이미지 저장
        with open('./'+search+'/'+search+'_'+str(i+1)+'_'+time.strftime('%y%m%d-%H%M')+'.jpg','wb') as h:
            img = f.read()
            h.write(img)